In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
sns.set(style='whitegrid', context='talk', palette='bright', color_codes=True)

from datetime import datetime

import re
import string
from collections import Counter

import pickle

In [2]:
transaction_data = pd.read_excel('QVI_transaction_data.xlsx')

In [3]:
transaction_data.head(10)

,DATE,STORE_NBR,LYLTY_CARD_NBR,TXN_ID,PROD_NBR,PROD_NAME,PROD_QTY,TOT_SALES
0,43390,1,1000,1,5,Natural Chip Compny SeaSalt175g,2,6.0
1,43599,1,1307,348,66,CCs Nacho Cheese 175g,3,6.3
2,43605,1,1343,383,61,Smiths Crinkle Cut Chips Chicken 170g,2,2.9
3,43329,2,2373,974,69,Smiths Chip Thinly S/Cream&Onion 175g,5,15.0
4,43330,2,2426,1038,108,Kettle Tortilla ChpsHny&Jlpno Chili 150g,3,13.8
5,43604,4,4074,2982,57,Old El Paso Salsa Dip Tomato Mild 300g,1,5.1
6,43601,4,4149,3333,16,Smiths Crinkle Chips Salt & Vinegar 330g,1,5.7
7,43601,4,4196,3539,24,Grain Waves Sweet Chilli 210g,1,3.6
8,43332,5,5026,4525,42,Doritos Corn Chip Mexican Jalapeno 150g,1,3.9
9,43330,7,7150,6900,52,Grain Waves Sour Cream&Chives 210G,2,7.2


In [4]:
transaction_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 264836 entries, 0 to 264835
Data columns (total 8 columns):
DATE              264836 non-null int64
STORE_NBR         264836 non-null int64
LYLTY_CARD_NBR    264836 non-null int64
TXN_ID            264836 non-null int64
PROD_NBR          264836 non-null int64
PROD_NAME         264836 non-null object
PROD_QTY          264836 non-null int64
TOT_SALES         264836 non-null float64
dtypes: float64(1), int64(6), object(1)
memory usage: 16.2+ MB


In [5]:
transaction_data.describe()

,DATE,STORE_NBR,LYLTY_CARD_NBR,TXN_ID,PROD_NBR,PROD_QTY,TOT_SALES
count,264836.000000,264836.00000,2.648360e+05,2.648360e+05,264836.000000,264836.000000,264836.000000
mean,43464.036260,135.08011,1.355495e+05,1.351583e+05,56.583157,1.907309,7.304200
std,105.389282,76.78418,8.057998e+04,7.813303e+04,32.826638,0.643654,3.083226
min,43282.000000,1.00000,1.000000e+03,1.000000e+00,1.000000,1.000000,1.500000
25%,43373.000000,70.00000,7.002100e+04,6.760150e+04,28.000000,2.000000,5.400000
50%,43464.000000,130.00000,1.303575e+05,1.351375e+05,56.000000,2.000000,7.400000
75%,43555.000000,203.00000,2.030942e+05,2.027012e+05,85.000000,2.000000,9.200000
max,43646.000000,272.00000,2.373711e+06,2.415841e+06,114.000000,200.000000,650.000000


In [6]:
transaction_data.isna().sum()

DATE              0
STORE_NBR         0
LYLTY_CARD_NBR    0
TXN_ID            0
PROD_NBR          0
PROD_NAME         0
PROD_QTY          0
TOT_SALES         0
dtype: int64

In [7]:
def convert_xl_time(xl_time):
    return pd.to_datetime('1900-01-01') + pd.to_timedelta(xl_time,'D')

In [8]:
# def xldate_to_datetime(xldate):
#     temp = datetime.datetime(1900, 1, 1)
#     delta = datetime.timedelta(days=xldate)
#     return temp + delta

In [9]:
transaction_data['DATE'] = transaction_data['DATE'].apply(convert_xl_time)

In [10]:
transaction_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 264836 entries, 0 to 264835
Data columns (total 8 columns):
DATE              264836 non-null datetime64[ns]
STORE_NBR         264836 non-null int64
LYLTY_CARD_NBR    264836 non-null int64
TXN_ID            264836 non-null int64
PROD_NBR          264836 non-null int64
PROD_NAME         264836 non-null object
PROD_QTY          264836 non-null int64
TOT_SALES         264836 non-null float64
dtypes: datetime64[ns](1), float64(1), int64(5), object(1)
memory usage: 16.2+ MB


In [11]:
transaction_data.head(10)

,DATE,STORE_NBR,LYLTY_CARD_NBR,TXN_ID,PROD_NBR,PROD_NAME,PROD_QTY,TOT_SALES
0,2018-10-19,1,1000,1,5,Natural Chip Compny SeaSalt175g,2,6.0
1,2019-05-16,1,1307,348,66,CCs Nacho Cheese 175g,3,6.3
2,2019-05-22,1,1343,383,61,Smiths Crinkle Cut Chips Chicken 170g,2,2.9
3,2018-08-19,2,2373,974,69,Smiths Chip Thinly S/Cream&Onion 175g,5,15.0
4,2018-08-20,2,2426,1038,108,Kettle Tortilla ChpsHny&Jlpno Chili 150g,3,13.8
5,2019-05-21,4,4074,2982,57,Old El Paso Salsa Dip Tomato Mild 300g,1,5.1
6,2019-05-18,4,4149,3333,16,Smiths Crinkle Chips Salt & Vinegar 330g,1,5.7
7,2019-05-18,4,4196,3539,24,Grain Waves Sweet Chilli 210g,1,3.6
8,2018-08-22,5,5026,4525,42,Doritos Corn Chip Mexican Jalapeno 150g,1,3.9
9,2018-08-20,7,7150,6900,52,Grain Waves Sour Cream&Chives 210G,2,7.2


In [12]:
transaction_data.groupby(['PROD_NAME']).sum()

,STORE_NBR,LYLTY_CARD_NBR,TXN_ID,PROD_NBR,PROD_QTY,TOT_SALES
PROD_NAME,,,,,,
Burger Rings 220g,211365,213917679,211659544,147016,2970,6831.0
CCs Nacho Cheese 175g,196830,197051283,196699043,98868,2839,5961.9
CCs Original 175g,206075,206366755,206359269,81756,2880,6048.0
CCs Tasty Cheese 175g,210177,210480069,210359656,140049,2890,6069.0
Cheetos Chs & Bacon Balls 190g,194220,194431594,193979011,26622,2801,9243.3
Cheetos Puffs 165g,197628,197844212,197677209,124528,2729,7641.2
Cheezels Cheese 330g,418716,419242617,418694255,72427,6017,34296.9
Cheezels Cheese Box 125g,194292,194574758,194289762,81424,2730,5733.0
Cobs Popd Sea Salt Chips 110g,439669,440229344,440060386,244875,6277,23852.6


In [13]:
transaction_data['words'] = [words for words in transaction_data['PROD_NAME']
                             .str.lower()
                             .str.replace('[{0}]*'.format(string.punctuation), '')
                             .str.strip()
                             .str.split()]

In [14]:
transaction_data

,DATE,STORE_NBR,LYLTY_CARD_NBR,TXN_ID,PROD_NBR,PROD_NAME,PROD_QTY,TOT_SALES,words
0,2018-10-19,1,1000,1,5,Natural Chip Compny SeaSalt175g,2,6.0,"[natural, chip, compny, seasalt175g]"
1,2019-05-16,1,1307,348,66,CCs Nacho Cheese 175g,3,6.3,"[ccs, nacho, cheese, 175g]"
2,2019-05-22,1,1343,383,61,Smiths Crinkle Cut Chips Chicken 170g,2,2.9,"[smiths, crinkle, cut, chips, chicken, 170g]"
3,2018-08-19,2,2373,974,69,Smiths Chip Thinly S/Cream&Onion 175g,5,15.0,"[smiths, chip, thinly, screamonion, 175g]"
4,2018-08-20,2,2426,1038,108,Kettle Tortilla ChpsHny&Jlpno Chili 150g,3,13.8,"[kettle, tortilla, chpshnyjlpno, chili, 150g]"
5,2019-05-21,4,4074,2982,57,Old El Paso Salsa Dip Tomato Mild 300g,1,5.1,"[old, el, paso, salsa, dip, tomato, mild, 300g]"
6,2019-05-18,4,4149,3333,16,Smiths Crinkle Chips Salt & Vinegar 330g,1,5.7,"[smiths, crinkle, chips, salt, vinegar, 330g]"
7,2019-05-18,4,4196,3539,24,Grain Waves Sweet Chilli 210g,1,3.6,"[grain, waves, sweet, chilli, 210g]"
8,2018-08-22,5,5026,4525,42,Doritos Corn Chip Mexican Jalapeno 150g,1,3.9,"[doritos, corn, chip, mexican, jalapeno, 150g]"
9,2018-08-20,7,7150,6900,52,Grain Waves Sour Cream&Chives 210G,2,7.2,"[grain, waves, sour, creamchives, 210g]"


In [15]:
transaction_data.columns

Index(['DATE', 'STORE_NBR', 'LYLTY_CARD_NBR', 'TXN_ID', 'PROD_NBR',
       'PROD_NAME', 'PROD_QTY', 'TOT_SALES', 'words'],
      dtype='object')

In [16]:
transaction_data = transaction_data[['DATE', 'STORE_NBR', 'LYLTY_CARD_NBR', 'TXN_ID', 'PROD_NBR',
       'PROD_NAME', 'words', 'PROD_QTY', 'TOT_SALES']]

In [17]:
transaction_data

,DATE,STORE_NBR,LYLTY_CARD_NBR,TXN_ID,PROD_NBR,PROD_NAME,words,PROD_QTY,TOT_SALES
0,2018-10-19,1,1000,1,5,Natural Chip Compny SeaSalt175g,"[natural, chip, compny, seasalt175g]",2,6.0
1,2019-05-16,1,1307,348,66,CCs Nacho Cheese 175g,"[ccs, nacho, cheese, 175g]",3,6.3
2,2019-05-22,1,1343,383,61,Smiths Crinkle Cut Chips Chicken 170g,"[smiths, crinkle, cut, chips, chicken, 170g]",2,2.9
3,2018-08-19,2,2373,974,69,Smiths Chip Thinly S/Cream&Onion 175g,"[smiths, chip, thinly, screamonion, 175g]",5,15.0
4,2018-08-20,2,2426,1038,108,Kettle Tortilla ChpsHny&Jlpno Chili 150g,"[kettle, tortilla, chpshnyjlpno, chili, 150g]",3,13.8
5,2019-05-21,4,4074,2982,57,Old El Paso Salsa Dip Tomato Mild 300g,"[old, el, paso, salsa, dip, tomato, mild, 300g]",1,5.1
6,2019-05-18,4,4149,3333,16,Smiths Crinkle Chips Salt & Vinegar 330g,"[smiths, crinkle, chips, salt, vinegar, 330g]",1,5.7
7,2019-05-18,4,4196,3539,24,Grain Waves Sweet Chilli 210g,"[grain, waves, sweet, chilli, 210g]",1,3.6
8,2018-08-22,5,5026,4525,42,Doritos Corn Chip Mexican Jalapeno 150g,"[doritos, corn, chip, mexican, jalapeno, 150g]",1,3.9
9,2018-08-20,7,7150,6900,52,Grain Waves Sour Cream&Chives 210G,"[grain, waves, sour, creamchives, 210g]",2,7.2


In [18]:
def get_words(df):
    all_words = []
    for l in df['words']:
        all_words.extend(l)
    return all_words

In [19]:
all_chip_words = get_words(transaction_data)

In [20]:
all_chip_words

['natural',
 'chip',
 'compny',
 'seasalt175g',
 'ccs',
 'nacho',
 'cheese',
 '175g',
 'smiths',
 'crinkle',
 'cut',
 'chips',
 'chicken',
 '170g',
 'smiths',
 'chip',
 'thinly',
 'screamonion',
 '175g',
 'kettle',
 'tortilla',
 'chpshnyjlpno',
 'chili',
 '150g',
 'old',
 'el',
 'paso',
 'salsa',
 'dip',
 'tomato',
 'mild',
 '300g',
 'smiths',
 'crinkle',
 'chips',
 'salt',
 'vinegar',
 '330g',
 'grain',
 'waves',
 'sweet',
 'chilli',
 '210g',
 'doritos',
 'corn',
 'chip',
 'mexican',
 'jalapeno',
 '150g',
 'grain',
 'waves',
 'sour',
 'creamchives',
 '210g',
 'smiths',
 'crinkle',
 'chips',
 'salt',
 'vinegar',
 '330g',
 'kettle',
 'sensations',
 'siracha',
 'lime',
 '150g',
 'twisties',
 'cheese',
 '270g',
 'ww',
 'crinkle',
 'cut',
 'chicken',
 '175g',
 'thins',
 'chips',
 'light',
 'tangy',
 '175g',
 'ccs',
 'original',
 '175g',
 'burger',
 'rings',
 '220g',
 'ncc',
 'sour',
 'cream',
 'garden',
 'chives',
 '175g',
 'doritos',
 'corn',
 'chip',
 'southern',
 'chicken',
 '150g',
 'c

In [21]:
all_chip_words_frequency = Counter(all_chip_words).most_common()

In [22]:
all_chip_words_frequency

[('175g', 60561),
 ('chips', 49770),
 ('150g', 43131),
 ('kettle', 41288),
 ('smiths', 28860),
 ('salt', 27976),
 ('cheese', 27890),
 ('pringles', 25102),
 ('134g', 25102),
 ('doritos', 24962),
 ('crinkle', 23960),
 ('110g', 22387),
 ('corn', 22063),
 ('original', 21560),
 ('cut', 20754),
 ('chip', 18645),
 ('170g', 18502),
 ('salsa', 18094),
 ('chicken', 15407),
 ('165g', 15297),
 ('sea', 14145),
 ('thins', 14075),
 ('chilli', 13895),
 ('sour', 13882),
 ('crisps', 12607),
 ('330g', 12540),
 ('vinegar', 12402),
 ('300g', 12041),
 ('rrd', 11894),
 ('infuzions', 11057),
 ('supreme', 10963),
 ('ww', 10320),
 ('cobs', 9693),
 ('popd', 9693),
 ('tortilla', 9580),
 ('tostitos', 9471),
 ('twisties', 9454),
 ('sensations', 9429),
 ('old', 9324),
 ('el', 9324),
 ('paso', 9324),
 ('dip', 9324),
 ('sweet', 7883),
 ('lime', 7852),
 ('chives', 7846),
 ('tomato', 7669),
 ('cream', 7618),
 ('thinly', 7507),
 ('tyrrells', 6442),
 ('380g', 6418),
 ('and', 6373),
 ('bbq', 6351),
 ('tangy', 6332),
 ('gra

In [23]:
def salsa_prod(df):
    return df[df['PROD_NAME'].str.contains("salsa", case=False)]

In [24]:
salsa_prod(transaction_data)

,DATE,STORE_NBR,LYLTY_CARD_NBR,TXN_ID,PROD_NBR,PROD_NAME,words,PROD_QTY,TOT_SALES
5,2019-05-21,4,4074,2982,57,Old El Paso Salsa Dip Tomato Mild 300g,"[old, el, paso, salsa, dip, tomato, mild, 300g]",1,5.1
25,2019-05-17,39,39144,35506,57,Old El Paso Salsa Dip Tomato Mild 300g,"[old, el, paso, salsa, dip, tomato, mild, 300g]",1,5.1
32,2019-05-22,45,45127,41122,64,Red Rock Deli SR Salsa & Mzzrlla 150g,"[red, rock, deli, sr, salsa, mzzrlla, 150g]",2,5.4
44,2018-08-20,56,56013,50090,39,Smiths Crinkle Cut Tomato Salsa 150g,"[smiths, crinkle, cut, tomato, salsa, 150g]",1,2.6
63,2019-05-17,82,82480,82047,101,Doritos Salsa Medium 300g,"[doritos, salsa, medium, 300g]",1,2.6
71,2018-08-17,94,94233,93956,65,Old El Paso Salsa Dip Chnky Tom Ht300g,"[old, el, paso, salsa, dip, chnky, tom, ht300g]",1,5.1
74,2018-08-18,97,97159,97271,35,Woolworths Mild Salsa 300g,"[woolworths, mild, salsa, 300g]",5,7.5
87,2018-08-17,116,116184,120270,59,Old El Paso Salsa Dip Tomato Med 300g,"[old, el, paso, salsa, dip, tomato, med, 300g]",1,5.1
114,2018-08-18,157,157185,159562,59,Old El Paso Salsa Dip Tomato Med 300g,"[old, el, paso, salsa, dip, tomato, med, 300g]",2,10.2
131,2018-08-21,175,175306,176634,59,Old El Paso Salsa Dip Tomato Med 300g,"[old, el, paso, salsa, dip, tomato, med, 300g]",1,5.1


In [25]:
transaction_data.drop(salsa_prod(transaction_data).index, inplace=True)

/Users/flatironschool/anaconda3/lib/python3.7/site-packages/pandas/core/frame.py:3940: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)


In [26]:
transaction_data

,DATE,STORE_NBR,LYLTY_CARD_NBR,TXN_ID,PROD_NBR,PROD_NAME,words,PROD_QTY,TOT_SALES
0,2018-10-19,1,1000,1,5,Natural Chip Compny SeaSalt175g,"[natural, chip, compny, seasalt175g]",2,6.0
1,2019-05-16,1,1307,348,66,CCs Nacho Cheese 175g,"[ccs, nacho, cheese, 175g]",3,6.3
2,2019-05-22,1,1343,383,61,Smiths Crinkle Cut Chips Chicken 170g,"[smiths, crinkle, cut, chips, chicken, 170g]",2,2.9
3,2018-08-19,2,2373,974,69,Smiths Chip Thinly S/Cream&Onion 175g,"[smiths, chip, thinly, screamonion, 175g]",5,15.0
4,2018-08-20,2,2426,1038,108,Kettle Tortilla ChpsHny&Jlpno Chili 150g,"[kettle, tortilla, chpshnyjlpno, chili, 150g]",3,13.8
6,2019-05-18,4,4149,3333,16,Smiths Crinkle Chips Salt & Vinegar 330g,"[smiths, crinkle, chips, salt, vinegar, 330g]",1,5.7
7,2019-05-18,4,4196,3539,24,Grain Waves Sweet Chilli 210g,"[grain, waves, sweet, chilli, 210g]",1,3.6
8,2018-08-22,5,5026,4525,42,Doritos Corn Chip Mexican Jalapeno 150g,"[doritos, corn, chip, mexican, jalapeno, 150g]",1,3.9
9,2018-08-20,7,7150,6900,52,Grain Waves Sour Cream&Chives 210G,"[grain, waves, sour, creamchives, 210g]",2,7.2
10,2019-05-19,7,7215,7176,16,Smiths Crinkle Chips Salt & Vinegar 330g,"[smiths, crinkle, chips, salt, vinegar, 330g]",1,5.7


In [27]:
salsa_prod(transaction_data)

,DATE,STORE_NBR,LYLTY_CARD_NBR,TXN_ID,PROD_NBR,PROD_NAME,words,PROD_QTY,TOT_SALES
